<a href="https://colab.research.google.com/github/wayne-chen1887/aps360-project/blob/discriminators-test/discriminators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import os 
import matplotlib.pyplot as plt

### **Global Discriminator:**


In [ ]:
"""
Input: 512 x 512

          kernel | stride | output 
conv1 --> 5 x 5  | 2 x 2  | 8  x 255 x 255
conv2 --> 5 x 5  | 2 x 2  | 16 x 126 x 126
conv3 --> 5 x 5  | 2 x 2  | 16 x 62  x 62
conv4 --> 5 x 5  | 2 x 2  | 32 x 30  x 30
conv5 --> 5 x 5  | 2 x 2  | 32 x 14  x 14
conv6 --> 5 x 5  | 2 x 2  | 64 x 6   x 6


          input        | output
fc    --> 64 x 6 x 6   | 1024
"""

In [138]:
class Global(nn.Module):
  def __init__(self):
    super(Global, self).__init__()
    self.main = nn.Sequential (
        nn.Conv2d(in_channels=3,  out_channels=8, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(8),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=2, padding=0),
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU()
    )
    self.fc1 = nn.Linear( 64*6*6, 1024)

  def forward(self, x):
    x = self.main(x)
    x = x.view(-1, 64*6*6)
    x = self.fc1(x)
    return x

### **Local Discriminator:**

In [ ]:
"""
Input: 64 x 64

          kernel | stride | output 
conv1 --> 5 x 5  | 2 x 2  | 8  x 31 x 31
conv2 --> 5 x 5  | 2 x 2  | 16 x 14 x 14
conv3 --> 5 x 5  | 1 x 1  | 32 x 10 x 10
conv4 --> 5 x 5  | 1 x 1  | 64 x 6  x 6

          input        | output
fc    --> 64 x 6 x 6   | 1024
""" 

In [142]:
class Local(nn.Module):
  def __init__(self):
    super(Local, self).__init__()
    self.main = nn.Sequential(
        nn.Conv2d(in_channels=3,  out_channels=8, kernel_size=5, stride=2, padding=1),
        nn.BatchNorm2d(8),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=2, padding=0),
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(64),
        nn.LeakyReLU()
    )
    self.fc1 = nn.Linear( 64*6*6, 1024)

  def forward(self, x):
    x = self.main(x)
    x = x.view(-1, 64*6*6)
    x = self.fc1(x)
    return x

### **Discriminator:**

In [159]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.name = "Discriminator"
    self.glo = Global()
    self.lo = Local()
    self.fc1 = nn.Sequential (
        nn.Linear(2048, 1),
        nn.Sigmoid(),
    )
  def forward(self, x, y):
    x = self.lo(x)
    print("local:", x.shape)
    y = self.glo(y)
    print("global:", y.shape)
    x = torch.cat((x, y), dim=1)
    print("concat:", x.shape)
    x = self.fc1(x)
    print("Result:", x)
    return x

### **Sample Data Loading:**

In [30]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [144]:
path = '/content/gdrive/My Drive/APS360_Labs/Discriminators/test'
transform = transforms.Compose([transforms.Resize((512,512)), 
                                transforms.ToTensor()])
transform2 = transforms.Compose([transforms.Resize((64,64)), 
                                transforms.ToTensor()])

globalset = torchvision.datasets.ImageFolder(path, transform=transform)
localset = torchvision.datasets.ImageFolder(path, transform=transform2)

global_loader = torch.utils.data.DataLoader(globalset, batch_size=1, shuffle=True, num_workers=0)
local_loader = torch.utils.data.DataLoader(localset, batch_size=1, shuffle=True, num_workers=0)

global_iter = iter(global_loader)
local_iter = iter(local_loader)

global_sample, label = global_iter.next()
local_sample, label1 = local_iter.next()



In [161]:
torch.manual_seed(1000)
model = Discriminator()
output = model(local_sample, global_sample)

local: torch.Size([1, 1024])
global: torch.Size([1, 1024])
concat: torch.Size([1, 2048])
Result: tensor([[0.5313]], grad_fn=<SigmoidBackward>)
